In [3]:
# 캐니 엣지 p.299
import numpy as np
import cv2

# "book.jpg" 이미지를 흑백 모드로 읽어옵니다.
src = cv2.imread("book.jpg", cv2.IMREAD_GRAYSCALE)

# Canny 엣지 감지 알고리즘을 사용하여 엣지를 검출합니다.
# - 첫 번째 인자(src): 입력 이미지, 흑백 이미지로 변환된 것
# - 두 번째 인자(100): 엣지를 검출하기 위한 최소 임계값
# - 세 번째 인자(200): 엣지를 검출하기 위한 최대 임계값
# - apertureSize: 엣지 검출에 사용되는 Sobel 연산 커널 크기
# - L2gradient: True로 설정하면 L2 노름을 사용하고, False로 설정하면 L1 노름을 사용합니다.
dst = cv2.Canny(src, 100, 200, apertureSize=3, L2gradient=True)

# 새로운 창을 생성하고 창의 크기를 조정합니다.
cv2.namedWindow("dst", flags=cv2.WINDOW_FREERATIO)
cv2.resizeWindow("dst", 1024, 768)

# "dst" 창에 엣지 이미지를 표시합니다.
cv2.imshow("dst", dst)

# 사용자가 키보드 입력을 기다립니다. 아무 키를 누를 때까지 창이 열려 있습니다.
cv2.waitKey(0)

# 모든 창을 닫습니다.
cv2.destroyAllWindows()


In [49]:
# 윤곽선 검출 p.309
import cv2

# "chess.png" 이미지를 읽어옵니다.
src = cv2.imread("chess.png")

# 입력 이미지를 복제하여 dst에 저장합니다.
dst = src.copy()

# MORPH_RECT 구조 요소를 생성합니다. 이것은 모폴로지 연산에 사용됩니다.
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

# 입력 이미지를 흑백 이미지로 변환합니다.
gray = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)

# 임계값을 설정하여 이진 이미지를 생성합니다.
# 이진 이미지는 검출하려는 객체를 흰색으로 나타냅니다.
ret, binary = cv2.threshold(gray, 230, 255, cv2.THRESH_BINARY)

# 모폴로지 연산을 수행하여 객체를 닫고, 노이즈를 제거합니다.
morp = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=2)

# 객체를 반전시켜서 검출된 객체를 흰색으로 유지합니다.
image = cv2.bitwise_not(morp)

# 객체의 외곽을 찾아내고 외곽선을 contours에 저장합니다.
contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

# 검출된 객체의 외곽에 빨간색 선을 그리고, 번호를 표시합니다.
for i in range(len(contours)):
    cv2.drawContours(dst, contours, -1, (0, 0, 255), 3)  # 빨간색 외곽선 그리기
    cv2.putText(dst, str(i), tuple(contours[i][0][0]), cv2.FONT_HERSHEY_COMPLEX, 1.3, (255, 0, 0), 1)  # 번호 표시
    print(i, hierarchy[0][i])  # 객체 번호와 계층 정보 출력

# 결과 이미지를 "dst" 창에 표시하고 사용자 입력을 대기합니다.
cv2.imshow("dst", dst)
cv2.waitKey(0)

# 창을 닫습니다.
cv2.destroyAllWindows()


0 [ 2 -1  1 -1]
1 [-1 -1 -1  0]
2 [ 4  0  3 -1]
3 [-1 -1 -1  2]
4 [ 8  2  5 -1]
5 [-1 -1  6  4]
6 [ 7 -1 -1  5]
7 [-1  6 -1  5]
8 [10  4  9 -1]
9 [-1 -1 -1  8]
10 [12  8 11 -1]
11 [-1 -1 -1 10]
12 [-1 10 13 -1]
13 [-1 -1 -1 12]


In [4]:
# 다각형 근사 p.314
import cv2

# "chess.png" 이미지를 읽어옵니다.
src = cv2.imread("chess.png")

# 입력 이미지를 복제하여 dst에 저장합니다.
dst = src.copy()

# MORPH_RECT 구조 요소를 생성합니다. 이것은 모폴로지 연산에 사용됩니다.
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

# 입력 이미지를 흑백 이미지로 변환합니다.
gray = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)

# 임계값을 설정하여 이진 이미지를 생성합니다.
# 이진 이미지는 검출하려는 객체를 흰색으로 나타냅니다.
ret, binary = cv2.threshold(gray, 230, 255, cv2.THRESH_BINARY)

# 모폴로지 연산을 수행하여 객체를 닫고, 노이즈를 제거합니다.
morp = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=2)

# 객체를 반전시켜서 검출된 객체를 흰색으로 유지합니다.
image = cv2.bitwise_not(morp)

# 객체의 외곽을 찾아내고 외곽선을 contours에 저장합니다.
contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

# 각 객체에 대해 다각형 근사화를 수행하고 다각형 주위에 빨간색 다각형을 그리고 꼭짓점을 파란색 원으로 표시합니다.
for i in contours:
    perimeter = cv2.arcLength(i, True)  # 객체의 둘레 길이 계산
    epsilon = perimeter * 0.05  # 근사화 정확도를 설정 (여기서는 둘레의 5%)
    approx = cv2.approxPolyDP(i, epsilon, True)  # 다각형 근사화 수행
    cv2.drawContours(dst, [approx], 0, (0, 0, 255), 3)  # 빨간색 다각형 그리기
    for j in approx:
        cv2.circle(dst, tuple(j[0]), 3, (255, 0, 0), -1)  # 꼭짓점 표시

# 결과 이미지를 "dst" 창에 표시하고 사용자 입력을 대기합니다.
cv2.imshow("dst", dst)
cv2.waitKey(0)

# 창을 닫습니다.
cv2.destroyAllWindows()


In [5]:
# 코너 검출 p.328
import cv2

# "dummy.jpg" 이미지를 읽어옵니다.
src = cv2.imread("dummy.jpg")

# 입력 이미지를 복제하여 dst에 저장합니다.
dst = src.copy()

# 입력 이미지를 흑백 이미지로 변환합니다.
gray = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)

# 흑백 이미지에서 코너(특징점)를 검출합니다.
# - corners: 검출된 코너 좌표
# - 100: 검출할 코너의 최대 개수
# - 0.01: 코너 검출 임계값
# - 5: 코너 간 최소 유효 거리
# - blockSize: 코너 검출에 사용되는 블록 크기
# - useHarrisDetector: Harris 코너 검출 방법 사용 여부
# - k: Harris 검출 방법에서의 k 값
corners = cv2.goodFeaturesToTrack(gray, 100, 0.01, 5, blockSize=3, useHarrisDetector=True, k=0.03)

# 검출된 코너 위치에 큰 파란색 원을 그립니다.
for i in corners:
    cv2.circle(dst, tuple(map(int, i[0])), 10, (255, 0, 0), cv2.FILLED)

# 코너 위치를 미세하게 조정합니다.
criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 30, 0.001)
cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), criteria)

# 조정된 코너 위치에 작은 빨간색 원을 그립니다.
for i in corners:
    cv2.circle(dst, tuple(map(int, i[0])), 5, (0, 0, 255), cv2.FILLED)

# 결과 이미지를 "dst" 창에 표시하고 사용자 입력을 대기합니다.
cv2.namedWindow("dst", flags=cv2.WINDOW_FREERATIO)
cv2.resizeWindow("dst", 1024, 768)
cv2.imshow("dst", dst)
cv2.waitKey(0)

# 창을 닫습니다.
cv2.destroyAllWindows()


In [6]:
# 직선 검출 p.336
import numpy as np
import cv2

# "card.jpg" 이미지를 읽어옵니다.
src = cv2.imread("card.jpg")

# 입력 이미지를 복제하여 dst에 저장합니다.
dst = src.copy()

# 직선 검출을 위한 커널 생성
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3), (-1, -1))

# 입력 이미지를 흑백 이미지로 변환합니다.
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

# 흑백 이미지를 이진 이미지로 변환합니다.
# 임계값을 150으로 설정하여 이진화합니다.
_, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

# 이진 이미지에 모폴로지 연산을 수행하여 노이즈를 제거합니다.
morp = cv2.dilate(binary, kernel)
morp = cv2.erode(morp, kernel, iterations=3)
morp = cv2.dilate(morp, kernel, iterations=2)

# Canny 엣지 검출을 수행하여 엣지 이미지 생성
canny = cv2.Canny(morp, 0, 0, apertureSize=3, L2gradient=True)

# 허프 변환을 사용하여 직선 검출
lines = cv2.HoughLines(canny, 1, np.pi/180, 140, srn=50, stn=10, min_theta=0, max_theta=np.pi/2)

# 검출된 직선을 이미지에 그리고, 직선의 끝점과 중심을 원으로 표시합니다.
for i in lines:
    rho, theta = i[0][0], i[0][1]
    a, b = np.cos(theta), np.sin(theta)
    x0, y0 = int(a * rho), int(b * rho)

    scale = src.shape[0] + src.shape[1]

    x1 = int(x0 + scale * -b)
    y1 = int(y0 + scale * a)
    x2 = int(x0 - scale * -b)
    y2 = int(y0 - scale * a)

    # 직선을 그리고 중심을 원으로 표시
    cv2.line(dst, (x1, y1), (x2, y2), (0, 255, 255), 2)
    cv2.circle(dst, (x0, y0), 3, (255, 0, 0), 5, cv2.FILLED)

# 결과 이미지를 "dst" 창에 표시하고 사용자 입력을 대기합니다.
cv2.namedWindow("dst", flags=cv2.WINDOW_FREERATIO)
cv2.resizeWindow("dst", 1024, 768)
cv2.imshow("dst", dst)
cv2.waitKey(0)

# 창을 닫습니다.
cv2.destroyAllWindows()


In [7]:
# 원 검출 p.343
import cv2

# "colorball.png" 이미지를 읽어옵니다.
src = cv2.imread("colorball.png")

# 입력 이미지를 복제하여 dst에 저장합니다.
dst = src.copy()

# 입력 이미지를 흑백 이미지로 변환합니다.
image = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

# Hough Circle 검출을 사용하여 이미지에서 원을 검출합니다.
# - cv2.HOUGH_GRADIENT: 원을 검출하기 위한 메소드로 그라디언트 방법을 사용
# - 1: 입력 이미지와 출력 이미지의 해상도 비율
# - 100: 물체 중심 사이의 최소 거리
# - param1=100: Canny 엣지 검출의 임계값 상한
# - param2=35: 원 검출의 임계값 (작을수록 더 많은 원 검출, 높을수록 정확한 원 검출)
# - minRadius=80: 원의 최소 반지름
# - maxRadius=120: 원의 최대 반지름
circles = cv2.HoughCircles(image, cv2.HOUGH_GRADIENT, 1, 100, param1=100, param2=35, minRadius=80, maxRadius=120)

# 검출된 원 주위에 흰색 원을 그립니다.
if circles is not None:
    circles = circles[0]  # circles 배열에서 원 정보 추출
    for i in circles:
        center = (int(i[0]), int(i[1]))  # 원의 중심 좌표
        radius = int(i[2])  # 원의 반지름
        cv2.circle(dst, center, radius, (255, 255, 255), 5)

# 결과 이미지를 "dst" 창에 표시하고 사용자 입력을 대기합니다.
cv2.namedWindow("dst", flags=cv2.WINDOW_FREERATIO)
cv2.resizeWindow("dst", 1024, 768)
cv2.imshow("dst", dst)
cv2.waitKey(0)

# 창을 닫습니다.
cv2.destroyAllWindows()
